In [92]:
import cobra
import csv
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis

model = cobra.io.load_json_model('e_coli_core.json')

filename = 'e_coli_core_expression.csv'
estimated_maximal_activity = {}

with open(filename, mode='r', newline='') as file:
    reader = csv.reader(file)   
    for row in reader:
        
        try:
            num = float(row[1])
            estimated_maximal_activity[row[0]] = num
        except ValueError:
            continue

high_absolute_default_bound = 1000

for reaction in model.reactions:
    
    if (reaction.id=="ATPM"):
        print("ATPM reaction unchanged")
        continue
    
    if (reaction.id=="EX_glc__D_e"):
        print("EX_glc__D_e bounds changed to high absolute default bound")
        reaction.lower_bound = -high_absolute_default_bound
        reaction.upper_bound = high_absolute_default_bound
        continue
    
    try:
        value = estimated_maximal_activity[reaction.id]
    except KeyError:
        continue
    
    if (reaction.reversibility == True):
        reaction.lower_bound = -value
        reaction.upper_bound = value
        
    elif (reaction.reversibility == False):
        reaction.lower_bound = 0.0
        reaction.upper_bound = value

ATPM reaction unchanged
EX_glc__D_e bounds changed to high absolute default bound


In [93]:
for reaction in model.reactions:
    print(f'{reaction.id}: UB:{reaction.upper_bound}, LB:{reaction.lower_bound}')

PFK: UB:12.12, LB:0.0
PFL: UB:1.0, LB:0.0
PGI: UB:13.12, LB:-13.12
PGK: UB:23.13, LB:-23.13
PGL: UB:8.12, LB:0.0
ACALD: UB:1.16, LB:-1.16
AKGt2r: UB:3.1, LB:-3.1
PGM: UB:20.01, LB:-20.01
PIt2r: UB:6.03, LB:-6.03
ALCD2x: UB:9.01, LB:-9.01
ACALDt: UB:2.29, LB:-2.29
ACKr: UB:1.19, LB:-1.19
PPC: UB:2.56, LB:0.0
ACONTa: UB:25.35, LB:-25.35
ACONTb: UB:25.35, LB:-25.35
ATPM: UB:1000.0, LB:8.39
PPCK: UB:25.23, LB:0.0
ACt2r: UB:3.23, LB:-3.23
PPS: UB:2.5, LB:0.0
ADK1: UB:30.57, LB:-30.57
AKGDH: UB:24.35, LB:0.0
ATPS4r: UB:60.5, LB:-60.5
PTAr: UB:4.47, LB:-4.47
PYK: UB:26.78, LB:0.0
BIOMASS_Ecoli_core_w_GAM: UB:1000.0, LB:0.0
PYRt2: UB:1.26, LB:-1.26
CO2t: UB:30.45, LB:-30.45
RPE: UB:5.67, LB:-5.67
CS: UB:20.56, LB:0.0
RPI: UB:5.56, LB:-5.56
SUCCt2_2: UB:2.36, LB:0.0
CYTBD: UB:40.56, LB:0.0
D_LACt2: UB:4.56, LB:-4.56
ENO: UB:28.78, LB:-28.78
SUCCt3: UB:6.67, LB:0.0
ETOHt2r: UB:2.34, LB:-2.34
SUCDi: UB:24.34, LB:0.0
SUCOAS: UB:20.6, LB:-20.6
TALA: UB:4.45, LB:-4.45
THD2: UB:4.5, LB:0.0
TKT1: UB:3

### Q3.c

We have 45 reactions which have a positive minimum flux. This is because some reaction are always needed for growth and survival. Thus these reaction will always have some flux. There are also exchange reaction, such as 'EX_co2_e' or 'EX_h2o_e'. These exchange reaction either provide a necessary chemical or remove unnecessary product.

In [94]:
# Perform FVA : finds the minimum and maximum flux for each reaction
fva_result: pd.DataFrame = flux_variability_analysis(model, model.reactions)
fva_result

,minimum,maximum
PFK,7.314456e+00,7.314456e+00
PFL,3.897323e-14,0.000000e+00
PGI,7.376438e+00,7.376438e+00
PGK,-1.446649e+01,-1.446649e+01
PGL,4.751035e-01,4.751035e-01
...,...,...
NADH16,2.026000e+01,2.026000e+01
NADTRHD,0.000000e+00,-4.232876e-16
NH4t,2.615007e+00,2.615007e+00
O2t,1.151629e+01,1.151629e+01


In [95]:
# Q3.b TODO
gene_imposed_reactions = [] # reactions with bounds imposed by gene expression data

for i, reaction_id in enumerate(fva_result.index):
    reaction = model.reactions.get_by_id(reaction_id)
    upper_bound = reaction.upper_bound # upper bound defined by the model
    max_flux = fva_result["maximum"].loc[reaction_id] # maximum flux found by FVA

    if upper_bound > max_flux:
        gene_imposed_reactions.append(reaction)


In [96]:
# Q3.c

pos_min_flux_reactions = fva_result[fva_result["minimum"] > 0].index # reactions with minimum flux > 0
pos_min_flux_reactions


Index(['PFK', 'PFL', 'PGI', 'PGL', 'AKGt2r', 'PIt2r', 'ACALDt', 'PPC',
       'ACONTa', 'ACONTb', 'ATPM', 'AKGDH', 'ATPS4r', 'PTAr', 'PYK',
       'BIOMASS_Ecoli_core_w_GAM', 'CS', 'CYTBD', 'ENO', 'SUCDi', 'TALA',
       'THD2', 'TKT1', 'TPI', 'EX_ac_e', 'EX_co2_e', 'EX_etoh_e', 'EX_h_e',
       'EX_h2o_e', 'EX_lac__D_e', 'EX_pyr_e', 'FBA', 'FUM', 'G6PDH2r', 'GAPD',
       'GLCpts', 'GLNS', 'GLNabc', 'GLUt2r', 'GND', 'ICDHyr', 'MDH', 'NADH16',
       'NH4t', 'O2t', 'PDH'],
      dtype='object')

In [ ]:
# 4.a
# Perform FBA optimization
fba_results = model.optimize()

# Report the maximal biomass production rate
print(f"Maximal biomass production rate: {fba_results.objective_value:.4f}")

Maximal biomass production rate: 0.4796


In [98]:
# 4.b: determine reaction whose flux reaches its maximum
reactions_at_max_flux = []
for reaction in model.reactions:
    flux = fba_results.fluxes[reaction.id]
    if flux == reaction.upper_bound:
        reactions_at_max_flux.append(reaction)

print(f"Number of reactions whose flux reaches its maximum: {len(reactions_at_max_flux)}")
print("Reactions whose flux reaches its maximum:")
for reaction in reactions_at_max_flux:
    print(f"- {reaction.id}")   

Number of reactions whose flux reaches its maximum: 2
Reactions whose flux reaches its maximum:
- THD2
- NADH16


In [103]:
#  4.c
print("Reactions with given maximal reaction activity and zero flux:")
list_4c = []
for reaction_id in estimated_maximal_activity.keys():
    reaction = model.reactions.get_by_id(reaction_id)
    if fba_results.fluxes[reaction.id] == 0:
        list_4c.append(reaction.id)
print(list_4c)


Reactions with given maximal reaction activity and zero flux:
['PFL', 'AKGt2r', 'ACALDt', 'PPCK', 'PPS', 'ADK1', 'SUCCt2_2', 'SUCCt3', 'FBP', 'FORt2', 'FORt', 'FRD7', 'FRUpts2', 'FUMt2_2', 'GLNabc', 'GLUN', 'GLUSy', 'GLUt2r', 'ICL', 'MALS', 'MALt2_2', 'ME1', 'ME2', 'NADTRHD']


The reaction we are going to analyze is 'PFL'. When we look at the reaction in the online demo we see that it is a reaction which only connects two point and runs in parallel with another reaction, 'PDH'. This also immediately explains why 'PFL' has no flux. All flux that is needed between the two nodes is provided by the other reaction. This make 'PFL' redundant. 

![image.png](Q4c.png)